In [ ]:
#필요한 라이브러리 정의
import re
import random
import os
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from webdriver_manager.chrome import ChromeDriverManager

from urllib.request import urlopen

import csv
import pandas as pd

# 반복작업을 생략하기 위해 만든 함수. 하지만 실제 사용은 하지 않음
def get_src():
    src=browser.page_source
    soup=BeautifulSoup(src,'lxml')
    return soup

# 검색란을 찾고 검색키를 보내는 함수
def search_keys():
    input_btn=browser.find_element_by_xpath('//*[@id="term"]')
    input_btn.send_keys('rRNA') # 이 부분이 검색키. 필요에 따라 수정 가능
    search_btn=browser.find_element_by_xpath('//*[@id="search"]')
    search_btn.click()

# 검색결과에서 균류 부분만 보는 함수
def get_fungi_db():
    src=browser.page_source
    soup=BeautifulSoup(src,'lxml')
    fungi_a=browser.find_element_by_xpath('//*[@id="_sp"]/li/ul/li[3]/a')
    fungi_a.click()

# 다음 페이지로 넘어가는 함수. 다른 방법을 찾아서 쓰지 않음.    
def get_more_info():
    next_page=browser.find_elements_by_class_name('active page_link next')
    for n in next_page:
        n.click()

def get_basic_info(): # 제목과 링크와 access 넘버를 받는 함수
    urls=[]
    titles=[]
    accessions=[]
    names=soup.find_all('p',attrs={'class':'title'}) # 제목(생물 종과 RNA 종류)을 모두 찾아 리스트 형태로 만듦
    accesses=soup.find_all('dl',attrs={'class':'rprtid'}) # 마찬가지로 리스트 형태로 만듦
    for i in range(len(names)): 
        name_p=names[i]
        title=name_p.get_text()
        titles.append(title)
        
        url="https://www.ncbi.nlm.nih.gov"+name_p.find('a')['href'] # href 부분 자료로 주소 합성
        urls.append(url)
        
        access=accesses[i].find('dd').get_text() # Accession 넘버는 <dl>태그 밑의 <dd> 태그에 존재
        accessions.append(access)
    return urls, titles, accessions

def get_fasta():
    Fastas=[]
    max_sleep_time=5 # 최대 지연 시간
    for i in range(len(urls)):
        browser.get(urls[i]) # urls의 원소를 browser에서 엶.
        src=browser.page_source
        soup=BeautifulSoup(src,'lxml')
        
        '''실제 사용자처럼 지연 시간을 랜덤으로 주는 함수. 최소 1초부터 최대 5초까지 임의로 할당함. 
        지연 시간을 주지 않으면 로딩 시간이 조금 길 경우에만 제대로 크롤링할 수 있음. 
        사이트에서 데이터 유출을 막아두기 때문에 지연 시간은 불필요해보여도 없애면 안 됨'''
        rand_value=random.randint(1,max_sleep_time) 
        time.sleep(rand_value)

        Fasta_btn=browser.find_element_by_id('ReportShortCut6')
        Fasta_btn.click()

        rand_value=random.randint(1,max_sleep_time) # 마찬가지로 지연시간
        time.sleep(rand_value)

        src=browser.page_source
        soup=BeautifulSoup(src,'lxml')

        try:
            Fasta=soup.find('pre').get_text() 
            Fastas.append(Fasta)
        except:
            Fasta='None' # try 부분에서 에러 발생 시 None 값 할당-> 불용데이터로 추후 지우거나 해야 함.
            Fastas.append(Fasta)

        print(i,Fasta) # 대용량 데이터 크롤링 시 작업 현황을 확인할 수 있도록 한 지표
    return Fastas

def count_none(): # 크롤링한 Fastas 리스트에서 불용 데이터 수 세기. 많으면 문제가 있다.
    cnt=0
    for f in Fastas:
        if 'None' in f:
            cnt+=1
    print(cnt)

In [ ]:
'''NCBI nucleotide 사이트로 접속'''
chrome_driver="C:/Users/82105/COS/chromedriver.exe"
browser=webdriver.Chrome(chrome_driver)
browser.implicitly_wait(3)
db="https://www.ncbi.nlm.nih.gov/nuccore"

browser.get(db)
src=browser.page_source
soup=BeautifulSoup(src,'lxml')

children은 부모보다 한 태그 아래에 있는 태그를 갖고 온다.
descendatns는 부모보다 아래에 있는 태그를 모두 갖고 온다.

In [ ]:
src=browser.page_source
soup=BeautifulSoup(src,'lxml')
search_keys()
get_fungi_db() 

In [ ]:
src=browser.page_source
soup=BeautifulSoup(src,'lxml')

'''20개씩 보기를 200개씩 보기로 바꿈'''
tw_per_page=browser.find_element_by_xpath('/html/body/div[1]/div[1]/form/div[1]/div[4]/div/div[1]/ul/li[2]/a')
tw_per_page.click()

src=browser.page_source
suop=BeautifulSoup(src,'lxml')

browser.find_element_by_xpath('//*[@id="ps200"]').click()

In [ ]:
'''기본 정보를 크롤링한다'''
src=browser.page_source
soup=BeautifulSoup(src,'lxml')

urls,titles,accessions=get_basic_info()

In [ ]:
Fastas=get_fasta()

In [ ]:
result_df=pd.DataFrame({'Species':titles, 'Url':urls, 'Accession':accessions, 'Fasta text':Fastas})
result_df.head(5)

In [ ]:
result_df.to_csv('Fungi rRNA Fasta Database.csv',encoding='cp949')

In [ ]:
fungi_df=pd.read_csv('Fungi rRNA Accession Database.csv',encoding='cp949')

In [ ]:
accession=fungi_df['Accession'].tolist()

with open을 쓰면 간단하게 메모장에 저장할 수 있다.

In [ ]:
with open('Fungi Accession.txt','w', encoding='utf-8') as f:
    for a in accession:
        f.write(a+'\n')